In [2]:
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser, MMCIFParser
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.SeqUtils import seq1
from Bio.Align import PairwiseAligner, substitution_matrices
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
def superimpose_and_plot(cif_file, pdb_file):
    
    def get_sequence(structure):
        return "".join(seq1(residue.get_resname()) for model in structure for chain in model for residue in chain if residue.id[0] == ' ')
    
    # Extract sequence from each file
    native = PDBParser(QUIET=True).get_structure("native", pdb_file)
    model = MMCIFParser(QUIET=True).get_structure("model", cif_file)
    
    native_seq = get_sequence(native)
    model_seq = get_sequence(model)

    # Perform global alignment using Needleman-Wunsch with BLOSUM62
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    blosum62 = substitution_matrices.load("BLOSUM62")
    aligner.substitution_matrix = blosum62
    alignments = aligner.align(native_seq, model_seq)

    # Take the best alignment
    alignment = alignments[0]

     # Access aligned sequences
    aligned_native = alignment.aligned[0]
    aligned_model = alignment.aligned[1]
    
    # Trim sequences based on alignment
    # Extract CA atoms for superimposition
    native_atoms = []
    model_atoms = []

    native_residues = [residue for model in native for chain in model for residue in chain if residue.id[0] == ' ']
    model_residues = [residue for model in model for chain in model for residue in chain if residue.id[0] == ' ']

    native_index, model_index = 0, 0
    for res_native, res_model in zip(aligned_native, aligned_model):
        if res_native != '-' and res_model != '-':
            native_atoms.append(native_residues[native_index]['CA'].get_coord())
            model_atoms.append(model_residues[model_index]['CA'].get_coord())
        if res_native != '-':
            native_index += 1
        if res_model != '-':
            model_index += 1

    # Convert lists to numpy arrays for SVD superimposition
    native_atoms = np.array(native_atoms)
    model_atoms = np.array(model_atoms)

    # Superimpose two structures using SVD
    super_imposer = SVDSuperimposer()
    super_imposer.set(native_atoms, model_atoms)
    super_imposer.run()

    # Extract RMSD
    rmsd = super_imposer.get_rms()
    print(f"RMSD: {rmsd:.3f} Å")

    return

In [10]:
pdb_file = 'af3_predictions/fold_t1124/7ux8.pdb'
cif_file = 'af3_predictions/fold_t1124/fold_t1124_model_0.cif'
superimpose_and_plot(cif_file, pdb_file)

RMSD: 0.015 Å
